# Collect and preprocess data

In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from fpdf import FPDF

# Directory paths
data_dir = "C:\\Users\\dalkh\\OneDrive\\Desktop\\saif folder\\project\\Real time monitoring construction activites project\construction image"

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 363 images belonging to 4 classes.


# Build a Convolutional Neural Network (CNN) Model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')  # Assuming 5 different stages of construction
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\dalkh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Train the Model

In [3]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
                validation_data=train_generator,
    validation_steps=50
)


Epoch 1/20


C:\Users\dalkh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 12/100 ━━━━━━━━━━━━━━━━━━━━ 1:36 1s/step - accuracy: 0.3081 - loss: 2.0535

C:\Users\dalkh\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 399ms/step - accuracy: 0.3061 - loss: 1.7972 - val_accuracy: 0.4435 - val_loss: 1.2575
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 343ms/step - accuracy: 0.4827 - loss: 1.1460 - val_accuracy: 0.5923 - val_loss: 0.9455
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 353ms/step - accuracy: 0.5465 - loss: 1.0472 - val_accuracy: 0.6198 - val_loss: 0.9056
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 358ms/step - accuracy: 0.5499 - loss: 0.9911 - val_accuracy: 0.6529 - val_loss: 0.8946
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 341ms/step - accuracy: 0.6303 - loss: 0.8698 - val_accuracy: 0.6639 - val_loss: 0.7536
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 359ms/step - accuracy: 0.6539 - loss: 0.8272 - val_accuracy: 0.6584 - val_loss: 0.8020
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 356ms/step - accuracy: 0.6560 - loss: 0.8125 - val_accuracy: 0.6749 - val_loss: 0.8344
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 353ms/step - accuracy: 0.6692 - loss: 0.7912 - val

# Evaluate the Model

In [4]:
loss, accuracy = model.evaluate(train_generator)
print(f"Accuracy: {accuracy*100:.2f}%")


12/12 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.7868 - loss: 0.5381
Accuracy: 79.89%


# Implement the Image Prediction Function

In [5]:
def predict_construction_stage(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0
    img = img.reshape(1, 150, 150, 3)
    
    prediction = model.predict(img)
    stage = train_generator.class_indices
    predicted_stage = max(stage, key=lambda key: prediction[0][stage[key]])
    
    return predicted_stage

# Object Detection


In [6]:
import cv2
import numpy as np

def detect_objects(image_path):
    # Load YOLO model
    net = cv2.dnn.readNet("C:\\Users\\dalkh\\OneDrive\\Desktop\\saif folder\\project\\Real time monitoring construction activites project\\yolo\\yolov3.weights", 
                           "C:\\Users\\dalkh\\OneDrive\\Desktop\\saif folder\\project\\Real time monitoring construction activites project\\yolo\\yolov3.cfg.cfg")

    # Load the COCO class labels
    with open("C:\\Users\\dalkh\\OneDrive\\Desktop\\saif folder\\project\\Real time monitoring construction activites project\\yolo\\coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    # Load image
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    
    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    detected_objects = []
    
    # Lists for detected bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []
    
    # Process each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # Only consider strong detections
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                # Append bounding box, confidence, and class ID
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply Non-Maxima Suppression to remove duplicates
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Draw bounding boxes and labels on the image
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            detected_objects.append(label)
            
            # Draw bounding box
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Label the detected object
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Display the image with detected objects
    cv2.imshow("Object Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return detected_objects


# Compare with Previous Images

In [7]:
def compare_with_previous(image_path, previous_stage):
    current_stage = predict_construction_stage(image_path)
    
    if current_stage is None:
        print("Error: Could not predict the current stage of construction.")
        return None
    
    if current_stage != previous_stage:
        print(f"Progress detected: Moved from {previous_stage} to {current_stage}.")
    else:
        print("No progress detected.")
    
    return current_stage


# Handle Errors

In [8]:
def validate_image(image_path, expected_stage):
    predicted_stage = predict_construction_stage(image_path)
    if predicted_stage != expected_stage:
        raise ValueError(f"Error: Uploaded image does not match the expected construction stage. Expected {expected_stage} but got {predicted_stage}.")
    return True


# Report Generation

In [9]:
def generate_report(image_path, expected_stage, current_stage, detected_objects, model_accuracy,image_description):
    report = FPDF()
    report.add_page()

    report.set_font("Arial", size=12)
    
    # Title
    report.set_font("Arial", size=16)
    report.cell(200, 10, txt="Construction Site Monitoring Report", ln=True, align="C")
    
    # Image Path
    report.set_font("Arial", size=12)
    report.cell(200, 10, txt=f"Image Path: {image_path}", ln=True)
    
    # Image Description
    report.cell(200, 10, txt="Image Description:", ln=True)
    report.multi_cell(0, 10, txt=image_description)
    
    # Expected vs Actual Stage
    report.cell(200, 10, txt=f"Expected Stage: {expected_stage}", ln=True)
    report.cell(200, 10, txt=f"Detected Stage: {current_stage}", ln=True)
    
    # Detected Objects
    report.cell(200, 10, txt="Detected Objects:", ln=True)
    for obj in detected_objects:
        report.cell(200, 10, txt=f"- {obj}", ln=True)
        
    # Model Accuracy
    report.cell(200, 10, txt=f"Model Accuracy: {model_accuracy*100:.2f}%", ln=True)    
           
    # Save Report
    report_output_path = "construction_report.pdf"
    report.output(report_output_path)
    print(f"Report generated: {report_output_path}")

In [10]:
image_description = "This image shows the facade stage of construction, where exterior finishing is taking place, and scaffolding is visible around the building."

# Input

In [ ]:
try:
    # Simulate the user uploading an image
    image_path = "C:\\Users\\dalkh\\OneDrive\\Desktop\\test foundation image.jpeg"
    expected_stage = "foundation"  # Example of what the user believes the stage to be

    # Validate the image against the expected stage
    validate_image(image_path, expected_stage)
    print(f"Image validated successfully as {expected_stage}.")
    
except ValueError as ve:
    print(ve)
    
finally:    
    # Compare with a previous image (for progress checking)
    previous_image_path = "C:\\Users\\dalkh\\OneDrive\\Desktop\\construction image\\foundation\\download (1).jpeg"
    previous_stage = predict_construction_stage(previous_image_path)
    
    # Compare the new image with the previous one
    current_stage = compare_with_previous(image_path, previous_stage)
    print(f"Current stage of construction: {current_stage}")
    
    # Object detection
    detected_objects = detect_objects(image_path)

  # Generate the report
    generate_report(image_path, expected_stage, current_stage, detected_objects, model_accuracy, image_description) 



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Error: Uploaded image does not match the expected construction stage. Expected foundation but got facade.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Progress detected: Moved from interior to facade.
Current stage of construction: facade
